In [1]:
library(BSgenome.Hsapiens.UCSC.hg38)
library(data.table)
for (i in names(Hsapiens)[1:24]){ # load all chromosome map files
        load(paste0('../sim-develop/data/root_maps/',i,'.rda'))
}
load('../sim-develop/data/exann.rda')

Loading required package: BSgenome
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, table, tapply, union, unique,
    unsplit, which, which.max, which.min

Loading required package: S4

In [2]:
head(exann)

chrom,start,end,geneSym
1,11869,12227,DDX11L1
1,12010,12057,DDX11L1
1,12179,12227,DDX11L1
1,12613,12721,DDX11L1
1,12613,12697,DDX11L1
1,12975,13052,DDX11L1


In [3]:
# Take the union of overlapping exonic ranges, and omit parent gene data
exann <- GRanges(exann$chrom,IRanges(exann$start,exann$end))
exann <- reduce(exann)
exann <- data.table(chrom=as.vector(seqnames(exann)),start=start(exann),end=end(exann))
head(exann)

chrom,start,end
1,11869,12227
1,12613,12721
1,12975,13052
1,13221,14501
1,15005,15038
1,15796,15947


#### In the cell below, we count the number of L1 target sites of each Snap-Velcro type in exons and non-exonic regions

In [4]:
counts <-  array(0,dim=c(24,8)) # Allocate matrix for counts
exon_lens <- rep(0,24)

#--- Loop through chromosome names
j<-1 # chromosome counter
for (i in names(Hsapiens)[1:24]){


        # Data objects containing indices of S-V sites for each category are labeled with the chromosome name.
        # Here we copy the data objects to a set of variables with generic names which can be used consistently 
        # in the loop. 
        map<-get(paste0(i,'Map')) # Get the site map data for the current chrom
        ict<-    map[[2]]
        icl<-    map[[3]]
        iot<-    map[[4]]
        iol<-    map[[5]]
        insites<-map[[1]] 
    
        i <- strsplit(i,"chr")[[1]][2] # Remove "chr" from chromosome name

################################################
        # count in exons for current chromosome (columns 1-4)
        ann_i <- exann[exann$chrom == i,] 
    
        sen        <-inrange(insites[ict[!is.na(ict[,1]),1],1],ann_i$start,ann_i$end) # Check if any Closed-Tight category sites are within the start-end range of ann_i
        antisen    <-inrange(insites[ict[!is.na(ict[,2]),2],2],ann_i$start,ann_i$end) 
        counts[j,1]<-length(which(sen)) + length(which(antisen))                      # Fill an element of the counts table with the count
        
        sen        <-inrange(insites[icl[!is.na(icl[,1]),1],1],ann_i$start,ann_i$end) 
        antisen    <-inrange(insites[icl[!is.na(icl[,2]),2],2],ann_i$start,ann_i$end)
        counts[j,2]<-length(which(sen)) + length(which(antisen))
        
        sen        <-inrange(insites[iot[!is.na(iot[,1]),1],1],ann_i$start,ann_i$end) 
        antisen    <-inrange(insites[iot[!is.na(iot[,2]),2],2],ann_i$start,ann_i$end)
        counts[j,3]<-length(which(sen)) + length(which(antisen))
        
        sen        <-inrange(insites[iol[!is.na(iol[,1]),1],1],ann_i$start,ann_i$end) 
        antisen    <-inrange(insites[iol[!is.na(iol[,2]),2],2],ann_i$start,ann_i$end)
        counts[j,4]<-length(which(sen)) + length(which(antisen))  
    
################################################
        # count in non-exons for current chromosome (columns 5-8)
        
        counts[j,5]   <- length(which(!is.na(as.vector(ict)))) - counts[j,1]
        counts[j,6]  <- length(which(!is.na(as.vector(icl)))) - counts[j,2]
        counts[j,7]  <- length(which(!is.na(as.vector(iot)))) - counts[j,3]
        counts[j,8]  <- length(which(!is.na(as.vector(iol)))) - counts[j,4]
    
        exon_lens[j] <- sum(ann_i$end-ann_i$start)
    
        j<-j+1
        rm('sen','antisen')
}

In [5]:
colnames(counts) <- c('ct_exon','cl_exon','ot_exon','ol_exon','ct_nonexon','cl_nonexon','ot_nonexon','ol_nonexon')
rownames(counts) <- names(Hsapiens)[1:24]
# counts

Here we account for male and female diploid genomes

#### Male

In [6]:
# Male genome - double the count for chromosomes 1:22
counts_dipl_m <- counts
counts_dipl_m[1:22,]<-counts_dipl_m[1:22,]*2
# counts_dipl_m

#### Female

In [7]:
# Female genome - double the counts for chroms. 1:23; count for chrom. Y is set to 0
counts_dipl_f <- counts
counts_dipl_f[1:23,]<-counts_dipl_f[1:23,]*2
counts_dipl_f[24,]<-counts_dipl_f[24,]*0
# counts_dipl_f

## Calculating probabilities of ENd (endonuclease-dependent) exonic vs non-exonic insertion

#### Male

In [8]:
counts_ovchr_m <- colSums(counts_dipl_m)
probs_ENd_m <- rep(0,2)
for (i in 1:2){
    probs_ENd_m[i] = 7.46*counts_ovchr_m[(i-1)*4+1] + 
                   3.48*counts_ovchr_m[(i-1)*4+2] + 
                   2.05*counts_ovchr_m[(i-1)*4+3] + 
                   1.00*counts_ovchr_m[(i-1)*4+4]
}
probs_ENd_m <- probs_ENd_m/sum(probs_ENd_m)
probs_ENd_m

[1] 0.03073668 0.96926332

#### Female

In [9]:
counts_ovchr_f <- colSums(counts_dipl_f)
probs_ENd_f <- rep(0,2)
for (i in 1:2){
    probs_ENd_f[i] = 7.46*counts_ovchr_f[(i-1)*4+1] + 
                   3.48*counts_ovchr_f[(i-1)*4+2] + 
                   2.05*counts_ovchr_f[(i-1)*4+3] + 
                   1.00*counts_ovchr_f[(i-1)*4+4]
}
probs_ENd_f <- probs_ENd_f/sum(probs_ENd_f)
probs_ENd_f

[1] 0.03049029 0.96950971

## Calculating probabilities of ENi (endonuclease-independent) exonic vs non-exonic insertion

In [10]:
chrlens <- rep(0,24)
for (i in 1:24){
    chrlens[i] <- length(Hsapiens[[i]])
}
sum(chrlens)

[1] 3088269832

#### Male

In [11]:
probs_ENi_m <- rep(0,2)
probs_ENi_m[1] <- (sum(exon_lens[1:22]*2)+sum(exon_lens[23:24]))/(sum(chrlens[1:22]*2)+sum(chrlens[23:24]))
probs_ENi_m[2] <- 1-probs_ENi_m[1]
probs_ENi_m

[1] 0.04277681 0.95722319

#### Female

In [12]:
probs_ENi_f <- rep(0,2)
probs_ENi_f[1] <- sum(exon_lens[1:23]*2)/sum(chrlens[1:23]*2)
probs_ENi_f[2] <- 1-probs_ENi_f[1]
probs_ENi_f

[1] 0.04271956 0.95728044

## Calculating final probabilities of exonic vs non-exonic insertion

#### Male

In [13]:
pd_exvsnon_m <- (.9*probs_ENd_m)+(0.1*probs_ENi_m) # Assuming 0.9 probability that insertion will be ENd and 0.1 for ENi
pd_exvsnon_m <- pd_exvsnon_m/sum(pd_exvsnon_m)
pd_exvsnon_m

[1] 0.03194069 0.96805931

#### Female

In [14]:
pd_exvsnon_f <- (.9*probs_ENd_f)+(0.1*probs_ENi_f) # Assuming 0.9 probability that insertion will be ENd and 0.1 for ENi
pd_exvsnon_f <- pd_exvsnon_f/sum(pd_exvsnon_f)
pd_exvsnon_f

[1] 0.03171322 0.96828678

In [15]:
lens <- cbind(exon_lens, chrlens)
lens

exon_lens,chrlens
12208560,248956422
9306870,242193529
7418406,198295559
5433490,190214555
6290982,181538259
6245624,170805979
6261771,159345973
4859198,145138636
4925084,138394717
4933365,133797422


In [16]:
save(lens,counts_dipl_m,counts_dipl_f,pd_exvsnon_m,pd_exvsnon_f,file='../sim-develop/data/exonicvsnon_counts.rda')